## Define Modified Arrhenius function
Outputs:
- k	=	rate constant

Takes in:
- A	=	pre-exponential factor
- n	=	Temperature dependence factor
- B	=	activation energy divided by universal gas constant ($E_a/R$)
- T	=	absolute temperature (in Kelvin)

In [9]:
import torch
def arrhenius(A,n,B,T):
    eps=1e-30 # Tolerance to avoid -inf  
    logk = torch.log10(torch.abs( A*torch.pow(T/300,n)*torch.exp(-B/(T)) )+eps)
    return logk

tensor(-inf)

## Plotting functions
Plots results

In [5]:
import matplotlib.pyplot as plt

def plot_loss(loss):
    plt.figure(0)
    plt.plot(loss['train'], label='Training loss')
    plt.plot(loss['valid'], label='Validation loss')
    plt.legend(bbox_to_anchor=(1.5, 1))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

def plot_results(actual,predicted,temperature):
    plt.figure(1)
    plt.plot(actual,predicted,'.')
    plt.plot(actual,actual)
    plt.title('Actual vs. predicted')
    plt.xlabel('Actual $log_{10}(k)$')
    plt.ylabel('Predicted $log_{10}(k)$')
    plt.figure(2)
    plt.plot(temperature,actual-predicted,'.')
    plt.title('Temperature dependent residual')
    plt.xlabel('Temperature [K]')
    plt.ylabel('Residual')

            
def plot_arrhenius_params(target,A,n,B):
    plt.figure(3)
    plt.plot(target[:,1],A.cpu().detach().numpy(),'.')
    plt.plot(target[:,1],target[:,1])
    plt.title('Pre-exponential factor (log-log scale)')
    plt.xlabel('Actual A')
    plt.ylabel('Predicted A')
    plt.figure(4)
    plt.plot(target[:,2],B.cpu().detach().numpy(),'.')
    plt.plot(target[:,2],target[:,2])
    plt.title('Normalized activation energy $B=E_a/R$')
    plt.xlabel('Actual B')
    plt.ylabel('Predicted B')
    plt.figure(5)
    plt.plot(target[:,3],n.cpu().detach().numpy(),'.')
    plt.plot(target[:,3],target[:,3])
    plt.title('Temperature dependence factor')
    plt.xlabel('Actual n')
    plt.ylabel('Predicted n')

In [1]:
import numpy as np

def calculate_error(target,Prediction):
    MAE = np.mean(np.abs(target-Prediction))
    RMS = np.sqrt(((target-Prediction)**2).mean())
    Rsq = np.corrcoef(target,Prediction)[0,1]**2
    return(MAE,RMS,Rsq)


def print_error(target,Prediction):

    if type(target)==torch.Tensor:
        target = target.detach().numpy()
    if type(Prediction)==torch.Tensor:
        Precition = Prediction.detach().numpy()

    if target.shape[1]>3:
        print('Prediction error for OH ; O3 ; NO3 ; Cl')
    MAE = np.zeros([target.shape[1]])
    RMS = np.zeros([target.shape[1]])
    Rsq = np.zeros([target.shape[1]])
    for i in range(target.shape[1]):
        MAE[i],RMS[i],Rsq[i] = calculate_error(target[~np.isnan(target[:,i]),i],
                                               Prediction[~np.isnan(target[:,i]),i])
    print('MAE',np.round(MAE,3))
    print('RMS',np.round(RMS,3))
    print('Rsq',np.round(Rsq,3))
    return(RMS)


In [ ]:
import random
def set_seed(seed):
    print("Set Seed to",seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False